In [ ]:
!pip install pandas openpyxl

In [ ]:
import os
import pandas as pd
import re

In [ ]:
os.chdir('/home/jovyan/work')

In [ ]:
ccg = pd.read_csv('data/ccg_lookup.csv')

In [ ]:
ccg_lad_lookup = pd.read_csv('data/ccg_lad_lookup.csv')

In [ ]:
population = pd.read_csv('data/ons-population-estimates.csv').drop(columns=[
    'CCG Name', 'STP21 Code', 'STP21 Name', 'NHSER21 Code', 'NHSER21 Name', 
])

In [ ]:
data = ccg.merge(
    population,
    how='left',
    left_on='CCG21CD',
    right_on='CCG Code',
).drop(
    columns=['CCG Code']
)

In [ ]:
origin = pd.read_excel(
    './source/Origins By Local Authority 2021.xlsx',
    na_filter=False
)

origin = origin[origin.LAD20CD.str.match('^E')].drop(
    columns = [
        'NWB%',
       'AAA : ENGLISH (Weighted)', 'BAA : SCOTTISH (Weighted)',
       'BAB : WELSH (Weighted)', 'BBA : NORTHERN IRISH', 'BBB : IRISH',
       'CAA : SPANISH', 'CBA : FILIPINO', 'CCA : PORTUGESE OR BRAZILIAN',
       'DAA : FRENCH OR WALLOON', 'DCA : DUTCH OR FLEMISH', 'DDA : GERMAN',
       'DHA : ITALIAN OR MALTESE', 'DZZ : SCANDINAVIAN', 'EIA : POLISH',
       'EJZ : CZECH OR SLOVAK', 'EKA : HUNGARIAN', 'ELZ : BALTIC STATES',
       'EMF : ALBANIAN', 'EMZ : FORMERLY YUGOSLAV', 'ENA : BULGARIAN',
       'EOA : ROMANIAN', 'EPZ : RUSSIAN OR UKRAINIAN',
       'FAZ : GREEK OR GREEK CYPRIOT', 'GAA : JEWISH', 'GBA : ARMENIAN',
       'HAA : BLACK CARIBBEAN (Weighted)', 'HBA : NIGERIAN', 'HCA : GHANAIAN',
       'HDZ : BLACK SOUTH AFRICAN', 'HED : ETHIOPIAN',
       'HZZ : OTHER BLACK AFRICAN', 'IAZ : NORTH AFRICAN MUSLIM',
       'IBZ : SOMALI', 'ICA: TURKISH', 'IFA : IRANIAN', 'IHZ : PAKISTANI',
       'IIZ : KASHMIRI', 'IKA : BANGLADESHI', 'IZZ : OTHER MUSLIM',
       'JAA : SIKH', 'KAA : HINDU INDIAN', 'KBA : TAMIL OR SRI LANKAN',
       'KCA : BANGLADESHI HINDU', 'LAA : JAPANESE', 'LAB : KOREAN',
       'LBA : MANDARIN CHINESE', 'LBB : CANTONESE CHINESE', 'LCA : VIETNAMESE',
       'LZZ : OTHER EAST ASIAN', 'UNKNOWN', 'XXX : UNCLASSIFIED AND OTHER',
       'YYY : NOT RECOGNISED'
    ]
)
origin = origin.merge(ccg_lad_lookup, how='right', left_on=['LAD20CD'], right_on=['LAD21CD'])
origin = origin.groupby('CCG21CD').agg(sum)
origin['NWB_pct'] = pd.Series(origin['NWB'] / origin['Population Total'] * 100).round(2)
origin.to_csv('data/origin.csv')

In [ ]:
data = data.merge(
    origin,
    how='left',
    on='CCG21CD'
)

In [ ]:
trust_lookup = pd.read_csv('data/trust_ccg_lookup.csv').drop(columns=['trust_name', 'ccg_name'])

In [ ]:
cmhs = pd.read_excel(
    'source/CMHS.xlsx'
).merge(
    trust_lookup, left_on='Trust Code', right_on='trust_code'
).merge(
    ccg, on='CCG21CDH'
).drop(
    columns=['trust_code', 'Trust Code', 'Trust Name', 'CCG21CDH', 'CCG21NM']
)

cmhs = cmhs.drop(
    columns=[x for x in cmhs.columns if not re.match("^(meanQ|CCG)", x)]
).groupby('CCG21CD').agg('mean')

In [ ]:
data = data.merge(
    cmhs,
    how='left',
    on='CCG21CD',
)

In [ ]:
data.to_json('data/data.json')